# Unified Storage Extension Demo

This notebook demonstrates the **Unified Storage Extension** - a multi-perceptron storage system with:
- **Unified lattice model**: AM, W, and metadata graphs all represented as lattice instances
- **Roaring bitmap compression**: 6-50x compression for HLLSets
- **Multi-perceptron support**: Different hash functions and configurations per perceptron
- **Content-addressable**: IICA compliant (Immutable, Idempotent, Content Addressable)

## Architecture

```text
┌──────────────────────────────────────────────────┐
│         Unified Storage Extension                │
├──────────────────────────────────────────────────┤
│  Perceptrons Registry                            │
│  ├─ data_perceptron_1 (SHA1, seed=42)            │
│  ├─ metadata_perceptron_1 (SHA1, seed=99)        │
│  └─ image_perceptron_1 (xxHash, seed=123)        │
├──────────────────────────────────────────────────┤
│  Unified Lattice Tables                          │
│  ├─ lattices (typed by perceptron)               │
│  ├─ lattice_nodes (JSON properties)              │
│  ├─ lattice_edges (JSON properties)              │
│  └─ hllsets (Roaring compressed)                 │
├──────────────────────────────────────────────────┤
│  Entanglement Layer                              │
│  ├─ entanglements (cross-perceptron)             │
│  └─ entanglement_mappings (φ: L₁ → L₂)           │
└──────────────────────────────────────────────────┘
```

In [1]:
# Imports
from core.extensions.unified_storage import UnifiedStorageExtension, PerceptronConfig
from core.hllset import HLLSet
from core.hrt import HRT
import json

## 1. Initialize Unified Storage

Create an in-memory database with unified schema.

In [2]:
# Create unified storage
storage = UnifiedStorageExtension(":memory:")

print("Unified Storage initialized")
print(f"Available: {storage.is_available()}")
print(f"Capabilities: {', '.join(storage.get_capabilities())}")

Unified Storage initialized
Available: True
Capabilities: multi_perceptron, roaring_compression, unified_lattice, am_storage, w_storage, metadata_storage, entanglement_storage, content_addressable


## 2. Register Multiple Perceptrons

Each perceptron can have different configurations:
- **Data perceptron**: Process text data with n-token algorithm
- **Metadata perceptron**: Process database metadata (tables, columns, FKs)
- **Image perceptron**: (Future) Process images with different hash function

In [3]:
# Register data perceptron
data_config = PerceptronConfig(
    perceptron_id="data_perceptron_1",
    perceptron_type="data",
    hash_function="sha1",
    hash_seed=42,
    config_dict={
        "n_tokens": 5,
        "p_bits": 14,
        "description": "Text data processing"
    }
)
storage.register_perceptron(data_config, "Data perceptron for text processing")

# Register metadata perceptron
metadata_config = PerceptronConfig(
    perceptron_id="metadata_perceptron_1",
    perceptron_type="metadata",
    hash_function="sha1",
    hash_seed=99,
    config_dict={
        "schema_aware": True,
        "p_bits": 12,
        "description": "Database metadata processing"
    }
)
storage.register_perceptron(metadata_config, "Metadata perceptron for database schemas")

print("Registered perceptrons:")
for p in storage.list_perceptrons():
    print(f"  {p['perceptron_id']}: {p['perceptron_type']} ({p['hash_function']}, seed={p['hash_seed']})")

Registered perceptrons:
  data_perceptron_1: data (sha1, seed=42)
  metadata_perceptron_1: metadata (sha1, seed=99)


## 3. Data Perceptron: Store AM and W Lattices

Process text data and store both adjacency matrix and weight lattices.

In [4]:
# Create sample token data for AM lattice
# In real usage, this would come from HRT or ManifoldOS ingestion

# Sample tokens and their transitions (bigrams)
text = "the quick brown fox jumps over the lazy dog the fox runs fast"
tokens = text.split()

# Build transitions
from collections import defaultdict
transitions = defaultdict(int)
for i in range(len(tokens) - 1):
    transitions[(tokens[i], tokens[i+1])] += 1

# Create token to index mapping
unique_tokens = sorted(set(tokens))
token_to_idx = {token: idx for idx, token in enumerate(unique_tokens)}

# Create token_lut (mapping indices to tokens)
token_lut = {(idx, 0): token for token, idx in token_to_idx.items()}

# Create AM cells
am_cells = [
    (token_to_idx[src], token_to_idx[tgt], freq)
    for (src, tgt), freq in transitions.items()
]

print(f"Created sample data:")
print(f"  Unique tokens: {len(unique_tokens)}")
print(f"  Transitions: {len(am_cells)}")
print(f"  Tokens: {unique_tokens}")

Created sample data:
  Unique tokens: 10
  Transitions: 12
  Tokens: ['brown', 'dog', 'fast', 'fox', 'jumps', 'lazy', 'over', 'quick', 'runs', 'the']


### 3.1 Store AM Lattice (Adjacency Matrix)

AM lattice represents token transitions:
- **Nodes**: Tokens (am_token type)
- **Edges**: Transitions with frequency weights (am_transition type)

In [5]:
# Store AM lattice
am_lattice_id = storage.store_am_lattice(
    perceptron_id="data_perceptron_1",
    am_cells=am_cells,
    token_lut=token_lut,
    dimension=len(unique_tokens)
)

print(f"Stored AM lattice: {am_lattice_id}")

# Query AM lattice
am_info = storage.get_lattice_info(am_lattice_id)
am_nodes = storage.get_lattice_nodes(am_lattice_id, node_type="am_token")
am_edges = storage.get_lattice_edges(am_lattice_id, edge_type="am_transition")

print(f"\nAM Lattice Info:")
print(f"  Type: {am_info['lattice_type']}")
print(f"  Dimension: {am_info['dimension']}")
print(f"  Nodes: {len(am_nodes)} tokens")
print(f"  Edges: {len(am_edges)} transitions")

# Show top transitions
print(f"\nTop 5 transitions (by frequency):")
for edge in am_edges[:5]:
    src = next(n for n in am_nodes if n['node_id'] == edge['source_node'])
    tgt = next(n for n in am_nodes if n['node_id'] == edge['target_node'])
    print(f"  {src['properties']['token']} → {tgt['properties']['token']}: {int(edge['weight'])}x")

Stored AM lattice: 43d66c7c39455ab305c1ab822383d19594368229

AM Lattice Info:
  Type: AM
  Dimension: 10
  Nodes: 10 tokens
  Edges: 12 transitions

Top 5 transitions (by frequency):
  the → quick: 1x
  quick → brown: 1x
  brown → fox: 1x
  fox → jumps: 1x
  jumps → over: 1x


### 3.2 Store W Lattice (Weight/HLLSet Lattice)

W lattice represents context relationships:
- **Nodes**: HLLSets at different positions (w_hllset type)
- **Edges**: Morphisms showing similarity (w_morphism type)

In [7]:
# Create basic HLLSets from tokens (simulating W lattice)
basic_hllsets = []

# Group tokens into small sets to create HLLSets
token_groups = [
    unique_tokens[:3],   # Group 0
    unique_tokens[2:5],  # Group 1 (overlap)
    unique_tokens[4:7],  # Group 2
    unique_tokens[6:9],  # Group 3
    unique_tokens[8:],   # Group 4
]

for idx, token_group in enumerate(token_groups):
    if token_group:  # Skip empty groups
        hllset = HLLSet.from_batch(token_group, p_bits=14)
        position = f"group_{idx}"
        basic_hllsets.append((idx, position, hllset))

print(f"Created {len(basic_hllsets)} basic HLLSets")

# Create morphisms (compare HLLSets using Jaccard similarity)
morphisms = []
for i in range(len(basic_hllsets)):
    for j in range(i+1, min(i+3, len(basic_hllsets))):  # Connect to next 2
        hll_i = basic_hllsets[i][2]
        hll_j = basic_hllsets[j][2]
        
        # Calculate Jaccard similarity (built-in method)
        similarity = hll_i.similarity(hll_j)
        
        # Estimate intersection size for metadata
        card_i = hll_i.cardinality()
        card_j = hll_j.cardinality()
        card_union = hll_i.union(hll_j).cardinality()
        intersection = max(0, card_i + card_j - card_union)
        
        if similarity > 0.01:  # Only store significant similarities
            morphisms.append((
                i, j, similarity,
                {"type": "context_similarity", "intersection": intersection}
            ))

print(f"Created {len(morphisms)} morphisms")

# Store W lattice
w_lattice_id = storage.store_w_lattice(
    perceptron_id="data_perceptron_1",
    basic_hllsets=basic_hllsets,
    morphisms=morphisms,
    dimension=len(basic_hllsets)
)

print(f"\nStored W lattice: {w_lattice_id}")

# Query W lattice
w_info = storage.get_lattice_info(w_lattice_id)
w_nodes = storage.get_lattice_nodes(w_lattice_id, node_type="w_hllset")
w_edges = storage.get_lattice_edges(w_lattice_id, edge_type="w_morphism")

print(f"\nW Lattice Info:")
print(f"  Type: {w_info['lattice_type']}")
print(f"  Dimension: {w_info['dimension']}")
print(f"  Nodes: {len(w_nodes)} HLLSets")
print(f"  Edges: {len(w_edges)} morphisms")

# Show top morphisms
print(f"\nTop morphisms (by similarity):")
for edge in w_edges[:min(5, len(w_edges))]:
    src = next(n for n in w_nodes if n['node_id'] == edge['source_node'])
    tgt = next(n for n in w_nodes if n['node_id'] == edge['target_node'])
    print(f"  {src['properties']['position']} ↔ {tgt['properties']['position']}: {edge['weight']:.3f}")

Created 5 basic HLLSets
Created 4 morphisms

Stored W lattice: 98a6c5f6e35d3e8f92958ebc22bcd8acc9beb4c8

W Lattice Info:
  Type: W
  Dimension: 5
  Nodes: 5 HLLSets
  Edges: 4 morphisms

Top morphisms (by similarity):
  group_3 ↔ group_4: 0.250
  group_0 ↔ group_1: 0.200
  group_1 ↔ group_2: 0.200
  group_2 ↔ group_3: 0.200


## 4. HLLSet Compression Statistics

Roaring bitmap compression reduces storage by 6-50x.

In [8]:
# Get compression statistics
stats = storage.get_storage_stats()

print("Overall Storage Statistics:")
print(f"  Perceptrons: {stats['perceptrons']}")
print(f"  Lattices: {stats['lattices']}")
print(f"  Nodes: {stats['nodes']}")
print(f"  Edges: {stats['edges']}")
print(f"  HLLSets: {stats['hllsets']}")
print(f"\nHLLSet Compression:")
print(f"  Original size: {stats['hllset_compression']['total_original']:,} bytes")
print(f"  Compressed size: {stats['hllset_compression']['total_compressed']:,} bytes")
print(f"  Average ratio: {stats['hllset_compression']['avg_compression_ratio']:.2f}x")
print(f"  Space saved: {(1 - stats['hllset_compression']['total_compressed']/stats['hllset_compression']['total_original'])*100:.1f}%")

# Show individual HLLSet stats
print(f"\nSample HLLSet compression:")
for i, (_, position, hllset) in enumerate(basic_hllsets[:3]):
    hllset_stats = storage.get_hllset_stats(hllset.name)
    if hllset_stats:
        print(f"  HLLSet {i} ({position}):")
        print(f"    Cardinality: {hllset_stats['cardinality']:.1f}")
        print(f"    Original: {hllset_stats['original_size']:,} bytes")
        print(f"    Compressed: {hllset_stats['compressed_size']:,} bytes")
        print(f"    Ratio: {hllset_stats['compression_ratio']:.2f}x")

Overall Storage Statistics:
  Perceptrons: 2
  Lattices: 2
  Nodes: 15
  Edges: 16
  HLLSets: 5

HLLSet Compression:
  Original size: 81,920 bytes
  Compressed size: 180 bytes
  Average ratio: 461.80x
  Space saved: 99.8%

Sample HLLSet compression:
  HLLSet 0 (group_0):
    Cardinality: 3.0
    Original: 16,384 bytes
    Compressed: 38 bytes
    Ratio: 431.00x
  HLLSet 1 (group_1):
    Cardinality: 3.0
    Original: 16,384 bytes
    Compressed: 38 bytes
    Ratio: 431.00x
  HLLSet 2 (group_2):
    Cardinality: 3.0
    Original: 16,384 bytes
    Compressed: 38 bytes
    Ratio: 431.00x


## 5. Metadata Perceptron: Database Schema as Lattice

Demonstrate how database metadata can be represented as a lattice:
- **Nodes**: Tables, columns (meta_table, meta_column types)
- **Edges**: Foreign keys, dependencies (meta_fk, meta_dep types)

In [9]:
from core.extensions.unified_storage import LatticeNode, LatticeEdge
from core.hllset import compute_sha1

# Create metadata lattice
metadata_lattice_id = storage.create_lattice(
    perceptron_id="metadata_perceptron_1",
    lattice_type="metadata",
    dimension=10,
    config={"schema": "ecommerce", "database": "main"}
)

print(f"Created metadata lattice: {metadata_lattice_id}")

# Add table nodes
tables = [
    {"name": "customers", "columns": ["id", "name", "email"], "rows": 10000},
    {"name": "orders", "columns": ["id", "customer_id", "total"], "rows": 50000},
    {"name": "products", "columns": ["id", "name", "price"], "rows": 1000},
    {"name": "order_items", "columns": ["id", "order_id", "product_id", "quantity"], "rows": 150000},
]

table_nodes = {}
for idx, table in enumerate(tables):
    node_id = compute_sha1(f"meta_table:metadata_perceptron_1:{table['name']}")
    
    node = LatticeNode(
        node_id=node_id,
        node_index=idx,
        node_type="meta_table",
        content_hash=compute_sha1(table['name']),
        cardinality=float(table['rows']),
        properties={
            "table_name": table['name'],
            "columns": table['columns'],
            "row_count": table['rows']
        }
    )
    
    storage.store_lattice_node(metadata_lattice_id, node)
    table_nodes[table['name']] = node_id

print(f"Added {len(tables)} table nodes")

# Add foreign key edges
foreign_keys = [
    ("orders", "customers", "customer_id", 1.0),
    ("order_items", "orders", "order_id", 0.95),
    ("order_items", "products", "product_id", 0.8),
]

for src_table, tgt_table, fk_column, strength in foreign_keys:
    edge_id = compute_sha1(f"meta_fk:{src_table}:{tgt_table}:{fk_column}")
    
    edge = LatticeEdge(
        edge_id=edge_id,
        source_node=table_nodes[src_table],
        target_node=table_nodes[tgt_table],
        edge_type="meta_fk",
        weight=strength,
        properties={
            "fk_column": fk_column,
            "constraint_type": "foreign_key"
        }
    )
    
    storage.store_lattice_edge(metadata_lattice_id, edge)

print(f"Added {len(foreign_keys)} foreign key edges")

# Query metadata lattice
meta_info = storage.get_lattice_info(metadata_lattice_id)
meta_nodes = storage.get_lattice_nodes(metadata_lattice_id, node_type="meta_table")
meta_edges = storage.get_lattice_edges(metadata_lattice_id, edge_type="meta_fk")

print(f"\nMetadata Lattice Info:")
print(f"  Type: {meta_info['lattice_type']}")
print(f"  Perceptron: {meta_info['perceptron_type']}")
print(f"  Schema: {meta_info['config']['schema']}")
print(f"  Tables: {len(meta_nodes)}")
print(f"  Foreign Keys: {len(meta_edges)}")

print(f"\nSchema Structure:")
for node in meta_nodes:
    table_name = node['properties']['table_name']
    row_count = node['properties']['row_count']
    columns = ", ".join(node['properties']['columns'])
    print(f"  📊 {table_name} ({row_count:,} rows)")
    print(f"     Columns: {columns}")

print(f"\nForeign Key Relationships:")
for edge in meta_edges:
    src = next(n for n in meta_nodes if n['node_id'] == edge['source_node'])
    tgt = next(n for n in meta_nodes if n['node_id'] == edge['target_node'])
    fk_col = edge['properties']['fk_column']
    print(f"  {src['properties']['table_name']}.{fk_col} → {tgt['properties']['table_name']} (strength: {edge['weight']:.2f})")

Created metadata lattice: c46cb930cfd098b71e18d26d54911e80e52e3454
Added 4 table nodes
Added 3 foreign key edges

Metadata Lattice Info:
  Type: metadata
  Perceptron: metadata
  Schema: ecommerce
  Tables: 4
  Foreign Keys: 3

Schema Structure:
  📊 customers (10,000 rows)
     Columns: id, name, email
  📊 orders (50,000 rows)
     Columns: id, customer_id, total
  📊 products (1,000 rows)
     Columns: id, name, price
  📊 order_items (150,000 rows)
     Columns: id, order_id, product_id, quantity

Foreign Key Relationships:
  orders.customer_id → customers (strength: 1.00)
  order_items.order_id → orders (strength: 0.95)
  order_items.product_id → products (strength: 0.80)


## 6. Multi-Perceptron Architecture

The unified storage now contains lattices from **two different perceptrons**:
1. **Data perceptron**: Text processing (AM + W lattices)
2. **Metadata perceptron**: Database schema (metadata lattice)

Each perceptron maintains its own hash morphism consistency while sharing the same storage infrastructure.

In [10]:
# Summary of multi-perceptron storage
all_perceptrons = storage.list_perceptrons()

print("Multi-Perceptron Storage Summary:")
print("=" * 60)

for perceptron in all_perceptrons:
    perc_id = perceptron['perceptron_id']
    
    # Get lattices for this perceptron
    lattices = storage.conn.execute("""
        SELECT lattice_id, lattice_type, dimension
        FROM lattices WHERE perceptron_id = ?
    """, [perc_id]).fetchall()
    
    print(f"\n{perceptron['perceptron_type'].upper()} PERCEPTRON: {perc_id}")
    print(f"  Hash: {perceptron['hash_function']} (seed={perceptron['hash_seed']})")
    print(f"  Description: {perceptron['description']}")
    print(f"  Lattices: {len(lattices)}")
    
    for lattice_id, lattice_type, dimension in lattices:
        node_count = storage.conn.execute("""
            SELECT COUNT(*) FROM lattice_nodes WHERE lattice_id = ?
        """, [lattice_id]).fetchone()[0]
        
        edge_count = storage.conn.execute("""
            SELECT COUNT(*) FROM lattice_edges WHERE lattice_id = ?
        """, [lattice_id]).fetchone()[0]
        
        print(f"    • {lattice_type} lattice: {node_count} nodes, {edge_count} edges, dim={dimension}")

print("\n" + "=" * 60)
print("\n✅ Unified storage successfully handles multiple perceptrons!")

Multi-Perceptron Storage Summary:

DATA PERCEPTRON: data_perceptron_1
  Hash: sha1 (seed=42)
  Description: Data perceptron for text processing
  Lattices: 2
    • AM lattice: 10 nodes, 12 edges, dim=10
    • W lattice: 5 nodes, 4 edges, dim=5

METADATA PERCEPTRON: metadata_perceptron_1
  Hash: sha1 (seed=99)
  Description: Metadata perceptron for database schemas
  Lattices: 1
    • metadata lattice: 4 nodes, 3 edges, dim=10


✅ Unified storage successfully handles multiple perceptrons!


## 7. Query Examples: Cross-Lattice Analysis

Demonstrate querying across different lattice types and perceptrons.

In [11]:
# Query 1: Find most frequent transitions in AM
print("Query 1: Top 10 token transitions (AM lattice)")
print("-" * 60)

top_transitions = storage.conn.execute("""
    SELECT 
        src.properties->>'token' as source_token,
        tgt.properties->>'token' as target_token,
        e.weight as frequency
    FROM lattice_edges e
    JOIN lattice_nodes src ON e.source_node = src.node_id
    JOIN lattice_nodes tgt ON e.target_node = tgt.node_id
    WHERE e.edge_type = 'am_transition'
    ORDER BY e.weight DESC
    LIMIT 10
""").fetchall()

for src, tgt, freq in top_transitions:
    print(f"  {src} → {tgt}: {int(freq)}x")

# Query 2: Find most similar HLLSet pairs in W
print("\nQuery 2: Most similar context pairs (W lattice)")
print("-" * 60)

similar_contexts = storage.conn.execute("""
    SELECT 
        src.properties->>'position' as pos1,
        tgt.properties->>'position' as pos2,
        e.weight as similarity
    FROM lattice_edges e
    JOIN lattice_nodes src ON e.source_node = src.node_id
    JOIN lattice_nodes tgt ON e.target_node = tgt.node_id
    WHERE e.edge_type = 'w_morphism'
    ORDER BY e.weight DESC
    LIMIT 5
""").fetchall()

for pos1, pos2, sim in similar_contexts:
    print(f"  {pos1} ↔ {pos2}: {sim:.3f}")

# Query 3: Find largest tables in metadata
print("\nQuery 3: Largest tables (metadata lattice)")
print("-" * 60)

largest_tables = storage.conn.execute("""
    SELECT 
        properties->>'table_name' as table_name,
        CAST(properties->>'row_count' AS INTEGER) as row_count
    FROM lattice_nodes
    WHERE node_type = 'meta_table'
    ORDER BY CAST(properties->>'row_count' AS INTEGER) DESC
""").fetchall()

for table, rows in largest_tables:
    print(f"  {table}: {rows:,} rows")

# Query 4: Find tables with most dependencies
print("\nQuery 4: Tables with most foreign key relationships")
print("-" * 60)

table_dependencies = storage.conn.execute("""
    SELECT 
        n.properties->>'table_name' as table_name,
        COUNT(e.edge_id) as fk_count
    FROM lattice_nodes n
    LEFT JOIN lattice_edges e ON n.node_id = e.source_node
    WHERE n.node_type = 'meta_table' AND e.edge_type = 'meta_fk'
    GROUP BY n.node_id, n.properties
    ORDER BY COUNT(e.edge_id) DESC
""").fetchall()

for table, fk_count in table_dependencies:
    print(f"  {table}: {fk_count} outgoing FK(s)")

Query 1: Top 10 token transitions (AM lattice)
------------------------------------------------------------
  quick → brown: 1x
  lazy → dog: 1x
  runs → fast: 1x
  the → fox: 1x
  fox → jumps: 1x
  the → lazy: 1x
  jumps → over: 1x
  the → quick: 1x
  fox → runs: 1x
  over → the: 1x

Query 2: Most similar context pairs (W lattice)
------------------------------------------------------------
  group_3 ↔ group_4: 0.250
  group_0 ↔ group_1: 0.200
  group_2 ↔ group_3: 0.200
  group_1 ↔ group_2: 0.200

Query 3: Largest tables (metadata lattice)
------------------------------------------------------------
  order_items: 150,000 rows
  orders: 50,000 rows
  customers: 10,000 rows
  products: 1,000 rows

Query 4: Tables with most foreign key relationships
------------------------------------------------------------
  order_items: 2 outgoing FK(s)
  orders: 1 outgoing FK(s)


## 8. Final Statistics

Complete storage summary.

In [12]:
final_stats = storage.get_storage_stats()

print("\n" + "=" * 70)
print("  UNIFIED STORAGE EXTENSION - FINAL STATISTICS")
print("=" * 70)

print(f"\n📊 Storage Contents:")
print(f"   • Perceptrons:        {final_stats['perceptrons']}")
print(f"   • Lattices:           {final_stats['lattices']}")
print(f"   • Lattice Nodes:      {final_stats['nodes']}")
print(f"   • Lattice Edges:      {final_stats['edges']}")
print(f"   • HLLSets (unique):   {final_stats['hllsets']}")

comp = final_stats['hllset_compression']
if comp['total_original'] > 0:
    savings_pct = (1 - comp['total_compressed'] / comp['total_original']) * 100
    print(f"\n💾 Compression Efficiency:")
    print(f"   • Original Size:      {comp['total_original']:,} bytes ({comp['total_original']/1024:.1f} KB)")
    print(f"   • Compressed Size:    {comp['total_compressed']:,} bytes ({comp['total_compressed']/1024:.1f} KB)")
    print(f"   • Average Ratio:      {comp['avg_compression_ratio']:.2f}x")
    print(f"   • Space Saved:        {savings_pct:.1f}%")

print(f"\n✅ Unified storage successfully demonstrated!")
print(f"   All lattice types (AM, W, metadata) stored in unified schema")
print(f"   Multiple perceptrons with independent hash morphisms")
print(f"   Roaring compression achieving {comp['avg_compression_ratio']:.1f}x reduction")

print("\n" + "=" * 70)

# Close storage
storage.close()
print("\n🔒 Storage closed")


  UNIFIED STORAGE EXTENSION - FINAL STATISTICS

📊 Storage Contents:
   • Perceptrons:        2
   • Lattices:           3
   • Lattice Nodes:      19
   • Lattice Edges:      19
   • HLLSets (unique):   5

💾 Compression Efficiency:
   • Original Size:      81,920 bytes (80.0 KB)
   • Compressed Size:    180 bytes (0.2 KB)
   • Average Ratio:      461.80x
   • Space Saved:        99.8%

✅ Unified storage successfully demonstrated!
   All lattice types (AM, W, metadata) stored in unified schema
   Multiple perceptrons with independent hash morphisms
   Roaring compression achieving 461.8x reduction


🔒 Storage closed


## Summary

This notebook demonstrated:

1. **Unified Schema**: All lattice types (AM, W, metadata) stored in same tables with type discrimination
2. **Multi-Perceptron Support**: Different perceptrons with independent configurations sharing storage
3. **Roaring Compression**: 6-50x compression for HLLSets using bitmap encoding
4. **Content-Addressable**: All artifacts identified by content hashes (IICA compliant)
5. **Flexible Queries**: SQL queries across lattice types and perceptrons
6. **Type Safety**: Node/edge types discriminate between different semantic meanings

### Next Steps

- **Entanglement Storage**: Cross-perceptron morphisms (φ: L₁ → L₂)
- **Metadata Perceptron**: Full implementation for database schema processing
- **Performance Testing**: Benchmark with large-scale data
- **Migration Tools**: Convert existing storage to unified schema